In [1]:
import pandas as pd


In [50]:
def showProfile(df) :
    col_profile = pd.concat([df.dtypes, df.isna().sum()/len(df)*100, df.sample().T], axis=1 ).reset_index()
    col_profile.columns = ['列名', '类型', '缺失比%','样例'] 
    print(df.shape)  
    return (col_profile)

In [52]:
# load data
code = pd.read_pickle('./ex_list_basic_20230529.pkl')
indu = pd.read_pickle('./em_code_indu.pkl')
df = pd.read_pickle('./yjbg2022q4.pkl.gz')

In [53]:
# select A share from all codes
code = code[code.market != '主板B股']
# 业绩报告数据从2010年Q1开始的。
# 而且并不是上市以后才有数据，上市前可能因为要披露3年一期，所以也有数据。
# 通过下面ly可以在后面选取上市以后的业绩报告数据，以免分析时出现当期还没有上市但有业绩的情况。
# 这个其实就是在分析是新发驱动还是增长驱动时有用，考虑不这删除这些数据。
code['ly'] = pd.to_datetime(code['list_day']).dt.year
# code['ly'] = code['ly'].apply(lambda x: x if x>2010 else 2010)

# 选出与报告期数相对应的代码，去除报告期还没有上市交易的代码还是有必要的。
code_sel = code[code['ly'] < 2023]

# 把行业并进去
code_sel = code_sel.merge(indu[['code', 'indu']])
showProfile(code_sel)

(5060, 7)


,列名,类型,缺失比%,样例
0,code,object,0.0,000932
1,name,object,0.0,华菱钢铁
2,list_day,object,0.0,1999-08-03
3,market,object,0.0,主板A股
4,ex,object,0.0,深交所
5,ly,int32,0.0,1999
6,indu,object,0.0,钢铁行业


In [7]:
code_sel.to_pickle('./mydata/ashare_basic.pkl.gz')

In [55]:
# rename cols
df.columns = ['code', 'name', 'eps', 'income', 'profit', 'equity_ps', 'roe','cffo', 'gmargin', 'pdate', 'qs']
# del name , reoder
df = df[['code', 'qs', 'eps', 'income', 'profit', 'equity_ps', 'roe','cffo', 'gmargin', 'pdate']]


In [58]:
# 保存一个完全的季度报告数据
yjbg_to_2022q4 = df.merge(code_sel[['code']])
yjbg_to_2022q4.to_pickle('./mydata/yjbg_to_2023q4.pkl.gz')
showProfile(yjbg_to_2022q4)

(193068, 10)


,列名,类型,缺失比%,样例
0,code,object,0.000000,600511
1,qs,object,0.000000,20180930
2,eps,float64,1.493774,1.2393
3,income,float64,0.068888,28777116460.43
4,profit,float64,0.012949,950487969.99
5,equity_ps,float64,3.764477,11.449706
6,roe,float64,2.528643,11.28
7,cffo,float64,3.637061,1.205631
8,gmargin,float64,2.346324,8.769204
9,pdate,object,0.000000,2019-10-24


In [60]:
# 保存一个完整的年度报告数据
yjbg_to_y2022 = yjbg_to_2022q4[yjbg_to_2022q4['qs'].str[4:] == '1231']
yjbg_to_y2022.to_pickle('./mydata/yjbg_to_y2022.pkl.gz')
showProfile(yjbg_to_y2022)

(56460, 10)


,列名,类型,缺失比%,样例
0,code,object,0.000000,601066
1,qs,object,0.000000,20161231
2,eps,float64,3.119022,0.81
3,income,float64,0.049593,13258770356.799999
4,profit,float64,0.019483,5259251675.2
5,equity_ps,float64,0.021254,5.02518
6,roe,float64,1.043216,18.1
7,cffo,float64,0.053135,0.673042
8,gmargin,float64,2.350337,NaN
9,pdate,object,0.000000,2018-05-30


In [67]:
# 把年度报告拆分成只包括代码、期数和数据的多个文件。
# 这样可以上传到github上，方便读取。
def split_yjbg(df):
    cols = df.columns
    for i in range(2, len(cols)):
        sel = df.iloc[:, [0,1,i]]
        fname = f'./mydata/yjbg_y2022_{cols[i]}.pkl.gz'
        sel.to_pickle(fname)
        print(sel.columns)
        print(sel.shape)


In [68]:
split_yjbg(yjbg_to_y2022)

Index(['code', 'qs', 'eps'], dtype='object')
(56460, 3)
Index(['code', 'qs', 'income'], dtype='object')
(56460, 3)
Index(['code', 'qs', 'profit'], dtype='object')
(56460, 3)
Index(['code', 'qs', 'equity_ps'], dtype='object')
(56460, 3)
Index(['code', 'qs', 'roe'], dtype='object')
(56460, 3)
Index(['code', 'qs', 'cffo'], dtype='object')
(56460, 3)
Index(['code', 'qs', 'gmargin'], dtype='object')
(56460, 3)
Index(['code', 'qs', 'pdate'], dtype='object')
(56460, 3)


上述数据整理以后，上传到github。
利用github可以通过raw网址来读取源文件的方式，可以实现云上读数据。

由于众所周知的原因，我希望国内的平台也能支持这样的操作。

- 可惜，试了一下gitee,并不行。不让访问。
- codeup,也不行。